# Засоби підготовки та аналізу даних
### ФБ-22 Загородній Ярослав
##### Лабораторна робота №5

#### Реалізуйте завдання 1 за допомогою сучасних графічних бібліотек на ваш вибір: Plotly, Bokeh, Altair тощо. Додайте декілька вікон для візуалізації замість одного, спадне меню (drop-down menu) та інші інтерактивні елементи на власний розсуд.

In [31]:
import numpy as np
import plotly.graph_objs as go
import dash
from dash import dcc, html, Input, Output, State

In [46]:
# Задаємо початкові значення параметрів гармоніки та шуму
A_default = 1.0
omega_default = 1.0
phi_default = 0.0
noise_level_default = 0.1

# Створюємо функцію для генерації даних гармоніки з накладеним шумом
def generate_data(A, omega, phi, noise_level):
    t = np.linspace(0, 10, 1000)
    y_clean = A * np.sin(omega * t + phi)
    noise = np.random.normal(0, noise_level, t.shape)
    y_noisy = y_clean + noise
    return t, y_noisy

# Створюємо функцію для оновлення графіку при зміні параметрів
def update_graph(A, omega, phi, noise_level):
    t, y = generate_data(A, omega, phi, noise_level)
    trace = go.Scatter(x=t, y=y, mode='lines', name='Noisy Harmonic Function')
    layout = go.Layout(title='Noisy Harmonic Function',
                       xaxis=dict(title='Time'),
                       yaxis=dict(title='Amplitude'))
    return {'data': [trace], 'layout': layout}



In [47]:
# Ініціалізуємо Dash додаток
app = dash.Dash(__name__)

# Створюємо макет інтерфейсу зі слайдерами, кнопкою та графіком
app.layout = html.Div([
    html.H1(style={'color': 'white'}, children=["Interactive Harmonic Function Plot"]),
    dcc.Graph(id='harmonic-graph', style={'width': '80%', 'height': '500px'}),
    html.Label(style={'color': 'white'}, children=["Amplitude (A):"]),
    dcc.Slider(id='amplitude-slider', min=0, max=2, step=0.1, value=A_default),
    html.Label(style={'color': 'white'}, children=["Angular Frequency (omega):"]),
    dcc.Slider(id='omega-slider', min=0, max=2, step=0.1, value=omega_default),
    html.Label(style={'color': 'white'}, children=["Phase (phi):"]),
    dcc.Slider(id='phi-slider', min=0, max=2*np.pi, step=0.1, value=phi_default),
    html.Label(style={'color': 'white'}, children=["Noise Level:"]),
    dcc.Slider(id='noise-slider', min=0, max=1, step=0.1, value=noise_level_default),
    html.Button('Reset', id='reset-button', n_clicks=0),
    dcc.Checklist(
        id='noise-checkbox',
        options=[{'label': 'Виключити шум', 'value': 'disable_noise'}],
        value=[],
        style={'color': 'white'}
    )
])

# Підключаємо зв'язки між інтерфейсом та функціоналом
@app.callback(
    Output('harmonic-graph', 'figure'),
    [Input('amplitude-slider', 'value'),
     Input('omega-slider', 'value'),
     Input('phi-slider', 'value'),
     Input('noise-slider', 'value'),
     Input('reset-button', 'n_clicks'),
     Input('noise-checkbox', 'value')]
)
def update_figure(A, omega, phi, noise_level, n_clicks, noise_checkbox):
    if n_clicks:
        return update_graph(A_default, omega_default, phi_default, noise_level_default)
    elif 'disable_noise' in noise_checkbox:
        return update_graph(A, omega, phi, 0)  # Вимкнути шум, якщо вибрано в чекбоксі
    else:
        return update_graph(A, omega, phi, noise_level)

# Функція для скидання значень слайдерів до початкових
@app.callback(
    [Output('amplitude-slider', 'value'),
     Output('omega-slider', 'value'),
     Output('phi-slider', 'value'),
     Output('noise-slider', 'value'),
     Output('noise-checkbox', 'value')],
    [Input('reset-button', 'n_clicks')]
)
def reset_sliders(n_clicks):
    if n_clicks:
        return A_default, omega_default, phi_default, noise_level_default, []
    else:
        raise dash.exceptions.PreventUpdate

# Запускаємо додаток
if __name__ == '__main__':
    app.run_server(debug=True)


#### Реалізуйте ваш власний фільтр, використовуючи виключно Python (а також numpy, але виключно для операцій з масивами numpy.ndarray). Застосуйте фільтр

In [48]:
def apply_custom_filter(y_noisy, threshold=0.5):
    y_filtered = np.where(np.abs(y_noisy) > threshold, y_noisy, 0)
    return y_filtered

In [49]:
# Створюємо функцію для оновлення графіку при зміні параметрів
def update_graph(A, omega, phi, noise_level, apply_filter=False):  # Додано аргумент apply_filter зі значенням за замовчуванням
    t, y = generate_data(A, omega, phi, noise_level)
    if apply_filter:  # Виконуємо фільтрацію, якщо apply_filter == True
        y = apply_custom_filter(y)
    trace = go.Scatter(x=t, y=y, mode='lines', name='Noisy Harmonic Function')
    layout = go.Layout(title='Noisy Harmonic Function',
                       xaxis=dict(title='Time'),
                       yaxis=dict(title='Amplitude'))
    return {'data': [trace], 'layout': layout}

In [50]:
# Ініціалізуємо Dash додаток
app = dash.Dash(__name__)


# Створюємо макет інтерфейсу зі слайдерами, кнопкою та графіком
app.layout = html.Div([
    html.H1(style={'color': 'white'}, children=["Interactive Harmonic Function Plot"]),
    dcc.Graph(id='harmonic-graph', style={'width': '80%', 'height': '500px'}),
    html.Label(style={'color': 'white'}, children=["Amplitude (A):"]),
    dcc.Slider(id='amplitude-slider', min=0, max=2, step=0.1, value=A_default),
    html.Label(style={'color': 'white'}, children=["Angular Frequency (omega):"]),
    dcc.Slider(id='omega-slider', min=0, max=2, step=0.1, value=omega_default),
    html.Label(style={'color': 'white'}, children=["Phase (phi):"]),
    dcc.Slider(id='phi-slider', min=0, max=2*np.pi, step=0.1, value=phi_default),
    html.Label(style={'color': 'white'}, children=["Noise Level:"]),
    dcc.Slider(id='noise-slider', min=0, max=1, step=0.1, value=noise_level_default),
    html.Button('Reset', id='reset-button', n_clicks=0),
    dcc.Checklist(
        id='noise-checkbox',
        options=[{'label': 'Виключити шум', 'value': 'disable_noise'}],
        value=[],
        style={'color': 'white'}
    ),
    dcc.Checklist(
        id='filter-checkbox',
        options=[{'label': 'Використовувати фільтр', 'value': 'apply_filter'}],
        value=[],
        style={'color': 'white'}
    )
])

@app.callback(
    Output('harmonic-graph', 'figure'),
    [Input('amplitude-slider', 'value'),
     Input('omega-slider', 'value'),
     Input('phi-slider', 'value'),
     Input('noise-slider', 'value'),
     Input('reset-button', 'n_clicks'),
     Input('noise-checkbox', 'value'),
     Input('filter-checkbox', 'value')]  # Додано вхід для чекбокса фільтрації
)
def update_figure(A, omega, phi, noise_level, n_clicks, noise_checkbox, filter_checkbox):
    if n_clicks:
        return update_graph(A_default, omega_default, phi_default, noise_level_default)
    elif 'disable_noise' in noise_checkbox:
        return update_graph(A, omega, phi, 0)  # Вимкнути шум, якщо вибрано в чекбоксі
    else:
        apply_filter = True if 'apply_filter' in filter_checkbox else False  # Визначаємо, чи використовувати фільтр
        return update_graph(A, omega, phi, noise_level, apply_filter)  # Передаємо параметр apply_filter у функцію оновлення графіку
# Функція для скидання значень слайдерів до початкових
@app.callback(
    [Output('amplitude-slider', 'value'),
     Output('omega-slider', 'value'),
     Output('phi-slider', 'value'),
     Output('noise-slider', 'value'),
     Output('noise-checkbox', 'value')],
    [Input('reset-button', 'n_clicks')]
)
def reset_sliders(n_clicks):
    if n_clicks:
        return A_default, omega_default, phi_default, noise_level_default, []
    else:
        raise dash.exceptions.PreventUpdate

# Запускаємо додаток
if __name__ == '__main__':
    app.run_server(debug=True)